In [41]:
import pandas as pd
import mysql.connector
import re
from collections import defaultdict, Counter
from nltk.tokenize import wordpunct_tokenize
from config import DB, USR, PWD

In [2]:
con = mysql.connector.connect(database=DB, user=USR, password=PWD)
cur = con.cursor()

## Новые данные

In [537]:
cards = pd.read_csv("./data/undone/tblCards.csv").drop(["аннотация", "термины", "zvuk", "zvuk2", "photo", "photo2"], axis=1)
cards.head(1)

,текст,ключевые слова,программа,вопрос,село,год,информант,информант2,информант3,информант4,собиратель,собиратель2,собиратель3,собиратель4,номер
0,"[А что за старичок, к которому вы водили мужа?...","Ребенок, Демонология, Говорить, Болезнь, Медиц...",XIII,9 а,"Смоленская обл., Велижский р-н, Погорелье",2013,БТМ,NaN,NaN,NaN,НВП,АБМ,ОВБ,ПН,1


In [5]:
keywords = pd.read_csv("./data/undone/tblKeyWords.csv")
keywords.head(1)

,ключевые слова,код
0,Аборт,1


In [55]:
# keywords["ключевые слова"].tolist()

In [6]:
collectors = pd.read_csv("./data/undone/tblSobirately.csv")
collectors.head(1)

,Код собирателя,ФИО собирателя,Номер
0,АБМ,Мороз Андрей Борисович,184


In [186]:
informators = pd.read_csv("./data/undone/tblInformators.csv")
informators.head(1)

,инициалы,ФИО,село,биография,порядковый номер информанта
0,НЕ,Надежда Егоровна,"Брянская обл., Злынковский р-н, Карпиловка",Прим. 1937 г.р.,9692


## Препроцессинг

In [538]:
cards.columns = ['text', 'keywords', 'qlist', 'questions', 'place', 'year', 'i1', 'i2', 'i3', 'i4', 'c1', 'c2', 'c3', 'c4', 'N']

In [539]:
cards["keywords"] = cards["keywords"].apply(lambda x: [x.strip().lower() for x in x.split(",")])

In [540]:
cards["informators"] = cards[["i1", "i2", "i3", "i4"]].apply(lambda x: x.dropna().tolist(), axis=1)
cards["collectors"] = cards[["c1", "c2", "c3", "c4"]].apply(lambda x: x.dropna().tolist(), axis=1)

In [541]:
cards["place"] = cards["place"].apply(lambda x: x.replace("обл", "область").replace("р-н", "район"))
cards["place"] = cards["place"].apply(lambda x: [i.strip() for i in re.split("[\.,]+", x) if i.strip()])

In [542]:
def parse_questions(x):
    orig = x
    x = re.sub("([0-9]+)([а-яa-z])", "\g<1> \g<2>", x)
    result = defaultdict(list)
    row = wordpunct_tokenize(x)
    cur_num = 0
    for r in row:
        if r.isdigit():
            if cur_num not in result and cur_num != 0:
                result[cur_num].append("")
            cur_num = int(r)
        elif r in {",", ".", ";", ".,"}:
            continue
        elif r.isalpha():
            if r not in result[cur_num]:
                result[cur_num].append(r)
            elif r == "доп":
                result[0].append(r)
        else:
            print(r)
    if cur_num not in result and cur_num != 0:
        result[cur_num].append("")
        
    final = []
    for q in result:
        for subq in result[q]:
            final.append((q, subq))
    return sorted(final)

In [543]:
cards["questions"] = cards["questions"].apply(lambda x: parse_questions(x))

In [544]:
cards["qlist"] = cards["qlist"].apply(lambda x: x.replace(" ", "").replace("а", "a").replace("Х", "X"))

In [545]:
cards["place"] = cards["place"].apply(tuple)

In [546]:
cards["place"] = cards["place"].apply(lambda x: ("Смоленская обл", "Велижский р-н", "Будница") if x == ("Будница", "Велижский р-н", "Смоленская обл") else x)

In [547]:
cards["place"] = cards["place"].apply(lambda x: (x[0], None, x[1]) if len(x) == 2 else x)

In [548]:
cards["region"] = cards["place"].apply(lambda x: x[0])
cards["ditrict"] = cards["place"].apply(lambda x: x[1])
cards["village"] = cards["place"].apply(lambda x: x[2])

In [549]:
cards["fin_questions"] = cards[["qlist", "questions"]].apply(lambda x: [(x["qlist"], *i) for i in x["questions"]], axis=1)

In [550]:
cards = cards[["text", "keywords", "place", "year", "N", "informators", "collectors", "region", "ditrict", "village", "fin_questions"]]

In [551]:
cards.head(2)

,text,keywords,place,year,N,informators,collectors,region,ditrict,village,fin_questions
0,"[А что за старичок, к которому вы водили мужа?...","[ребенок, демонология, говорить, болезнь, меди...","(Смоленская область, Велижский район, Погорелье)",2013,1,[БТМ],"[НВП, АБМ, ОВБ, ПН]",Смоленская область,Велижский район,Погорелье,"[(XIII, 9, а)]"
1,[БТМ угощает собирателей у себя дома. АБМ попе...,"[пища, еда, угощение, приметы, поговорка]","(Смоленская область, Велижский район, Погорелье)",2013,2,[БТМ],"[НВП, АБМ, ОВБ, ВК]",Смоленская область,Велижский район,Погорелье,"[(XXV, 23, )]"


## данные из базы

In [552]:
cur.execute("SELECT question_list, question_num, question_letter, id FROM questions")
dbquestions = {(ql, qn, qlt): i for ql, qn, qlt, i in cur.fetchall()}

In [553]:
cur.execute("SELECT word, id FROM keywords")
dbkeywords = {word.lower().strip(): i for word, i in cur.fetchall()}

In [554]:
newkwd = set(keywords["ключевые слова"].apply(lambda x: x.strip().lower()).tolist()) - set(dbkeywords)

In [555]:
df = cards.copy()

In [244]:
cnt = defaultdict(int)
cnt_new = defaultdict(int)
for kwrds in df["keywords"].values:
    no = []
    for w in kwrds:
        if not w:
            continue
        elif w in dbkeywords:
            pass
        elif w in newkwd:
            cnt_new[w] += 1
        else:
            cnt[w] += 1

In [245]:
for row in sorted(cnt.items(), key=lambda x: x[1], reverse=True):
    print(*row[::-1], sep = "\t")

12	прозвища
11	варвара св.
7	звон
6	потоп
5	сказки
4	птица
4	шитье
2	пещера
2	кожа
2	гадание
2	язык
2	руки
1	рыбы
1	воскрес
1	огурец
1	терять–находить
1	кол
1	предания.
1	книг
1	лопата
1	молодой-ста
1	конец петра и павла день
1	муравейник
1	животные
1	михаил св.
1	удавленник
1	свадебный поезд
1	козел
1	полнолуние
1	крещенье (календ.)
1	старый-новый
1	ласка
1	смола
1	легенда
1	освящать
1	застолье
1	закрытый
1	знамение
1	правы-левый
1	границы
1	хозяин
1	умываться


In [66]:
for row in sorted(cnt_new.items(), key=lambda x: x[1], reverse=True):
    print(*row[::-1], sep = "\t")

59	смерть
28	нечистая сила
22	непогода
15	угол дома
8	предметы-обереги
6	коляда
5	новый
4	инородец
3	новобрачные
2	иноверец
2	доля
2	булочка
1	май
1	зелень


## Обработка данных

### Таблица тексты

In [67]:
cur.execute("SELECT max(id) FROM texts")
cur.fetchall()

[(7294,)]

In [556]:
df["id"] = list(range(7295, 7295 + df.shape[0]))
df["leader"] = "АБМ"

In [557]:
df["txtlen"] = df["text"].apply(len)

In [248]:
long = df[["id", "txtlen"]].sort_values(by="txtlen", ascending=False).head(100)
print(*long[long["txtlen"] > 9000]["id"].values.tolist(), sep=", ")

7763, 7706, 8145, 8149, 7759, 7543, 8260, 8261, 7315, 7467, 7753, 8817, 8052, 7630, 7786, 8421, 8505, 8247, 8355, 7928, 7796, 8053, 9136, 8490, 8283, 8259, 8472, 8679, 8467, 7466, 7305, 7306, 7872, 7959, 9333, 9342, 9375, 9392, 8508, 9326, 8169, 9373, 9394, 9331, 9368, 9367, 9393, 9378, 9332, 9379, 7560


In [558]:
data = df[["id", "text", "year", "leader"]].values.tolist()

In [99]:
# cur.executemany("INSERT INTO texts (id, raw_text, year, leader) VALUES (%s, %s, %s, %s)", data)
# con.commit()

### Вопросы

In [113]:
new_question = set()
for row in df["fin_questions"].values:
    no = []
    for q in row:
#         if q[1] == 0:
#             q = (q[0], q[1], "")
        if q not in dbquestions:
            no.append(q)
    if no:
        for n in no:
            new_question.add(n)

In [116]:
cur.executemany("INSERT INTO questions (question_list, question_num, question_letter) VALUES (%s, %s, %s)", list(sorted(new_question)))
con.commit()

In [118]:
cur.execute("SELECT question_list, question_num, question_letter, id FROM questions")
dbquestions = {(ql, qn, qlt): i for ql, qn, qlt, i in cur.fetchall()}

In [121]:
result = []

new_question = set()
for i, qs in df[["id", "fin_questions"]].values:
    for q in qs:
        result.append((i, dbquestions[q]))

In [123]:
cur.executemany("INSERT INTO t_q (id_text, id_question) VALUES (%s, %s)", result)
con.commit()

### Собиратели

In [126]:
cur.execute("SELECT code, id FROM collectors")
dbcollectors = {code: i for code, i in cur.fetchall()}

In [136]:
clc_mapping = {
    "Е. М. Боганева": "БЕМ", "КВA": "КВА", "ОБВ": "ОВБ"
}

In [145]:
clc = set()
no = set()
for i in df["collectors"].values:
    for j in i:
        j = j.strip(".")
        if j not in dbcollectors:
            no.add(j)

In [146]:
no

{',', 'Е. М. Боганева', 'КВA', 'ОБВ'}

In [142]:
collectors["code"] = collectors["Код собирателя"].apply(dbcollectors.get)

In [149]:
result = []

for n, i in df[["id", "collectors"]].values:
    for j in i:
        j = j.strip(".")
        j = clc_mapping[j] if j in clc_mapping else j
        if j in dbcollectors:
            result.append((n, dbcollectors[j]))
        else:
            print(j)

,


In [151]:
cur.executemany("INSERT INTO t_c (id_text, id_collector) VALUES (%s, %s)", result)
con.commit()

### Информанты

In [559]:
# cur.execute("SELECT code, id, name FROM informators")
# dbnameinf = {(code, n): i for code, i, n in cur.fetchall()}

# # cur.execute("SELECT code, id FROM informators")
# # dbinformators = {code: i for code, i in cur.fetchall()}

dbinf = pd.read_sql_query("SELECT id, code, name, current_region FROM informators", con=con)

In [560]:
dfinformators = dict(informators[["инициалы", "ФИО"]].values)

In [562]:
no = set()
for i in df["informators"].values:
    for j in i:
        j = j.strip(". ?")
        if j and j not in dbinformators and j not in dfinformators:
            no.add(j)

In [564]:
informators["db_name"] = informators[["инициалы", "ФИО"]].apply(lambda x: dbnameinf.get((x["инициалы"], x["ФИО"])), axis=1)

In [565]:
informators["place"] = informators["село"].apply(lambda x: x.replace("обл", "область").replace("р-н", "район"))
informators["place"] = informators["place"].apply(lambda x: [i.strip() for i in re.split("[\.,]+", x) if i.strip()])
informators["place"] = informators["place"].apply(tuple)
informators["place"] = informators["place"].apply(lambda x: (x[0], None, x[1]) if len(x) == 2 else x)

In [225]:
# informators["place"] = informators["place"].apply(lambda x: ("Смоленская обл", "Велижский р-н", "Будница") if x == ("Будница", "Велижский р-н", "Смоленская обл") else x)

In [566]:
informators["region"] = informators["place"].apply(lambda x: x[0])
informators["ditrict"] = informators["place"].apply(lambda x: x[1])
informators["village"] = informators["place"].apply(lambda x: x[2])

In [592]:
informators = informators.merge(dbinf, right_on=["code", "current_region"], left_on=["инициалы", "region"], how="left")

In [593]:
informators[informators["инициалы"] == "БАГ"]

,инициалы,ФИО,село,биография,порядковый номер информанта,db_name,place,region,ditrict,village,...,name_x,current_region_x,id_y,code_y,name_y,current_region_y,id,code,name,current_region
567,БАГ,Бордюков Александр Григорьевич,"Смоленская обл., Велиж","1928 г.р., род. в г. Велиж. Всю жизнь живет в ...",9980,NaN,"(Смоленская область, None, Велиж)",Смоленская область,None,Велиж,...,NaN,NaN,476.0,БАГ,Бордюков Александр Григорьевич,Смоленская область,476.0,БАГ,Бордюков Александр Григорьевич,Смоленская область


In [569]:
df["region"] = df["region"].replace("Будница", "Смоленская область")
df["village"] = df["village"].replace("Смоленская область", "Будница")

In [590]:
result = []
for i in range(df.shape[0]):
    row = df.iloc[i]
    
    for code in row["informators"]:
        code = code.strip().strip(".")
        if code == "???":
            continue
        infs = informators[informators["инициалы"] == code]
        if infs.shape[0] != 1:
#             print(code)
            infs = infs[infs["region"] == row["region"]]
            if infs.shape[0] != 1:
                print(row["id"], row["region"], code, infs["region"].tolist(), infs.shape)
            else:
                result.append((row["id"], infs["db_name"].values[0]))

7669 Смоленская область СМТ [] (0, 18)
7670 Смоленская область СМТ [] (0, 18)
7671 Смоленская область СМТ [] (0, 18)
7672 Смоленская область СМТ [] (0, 18)
7673 Смоленская область СМТ [] (0, 18)
7674 Смоленская область СМТ [] (0, 18)
7675 Смоленская область СМТ [] (0, 18)
7676 Смоленская область СМТ [] (0, 18)
7677 Смоленская область СМТ [] (0, 18)
7678 Смоленская область СМТ [] (0, 18)
7679 Смоленская область СМТ [] (0, 18)
7680 Смоленская область СМТ [] (0, 18)
7681 Смоленская область СМТ [] (0, 18)
7682 Смоленская область СМТ [] (0, 18)
7683 Смоленская область СМТ [] (0, 18)
7684 Смоленская область СМТ [] (0, 18)
7685 Смоленская область СМТ [] (0, 18)
7686 Смоленская область СМТ [] (0, 18)
7687 Смоленская область СМТ [] (0, 18)
7725 Смоленская область ВА [] (0, 18)
7725 Смоленская область М [] (0, 18)
7751 Могилевская область КАТ [] (0, 18)
7992 Могилевская область Х [] (0, 18)
8329 Могилевская область С ['Могилевская область', 'Могилевская область', 'Могилевская область', 'Могилевс

In [574]:
new_informators = informators[(informators["id"].isna()) & (informators["region"] != "Псковская область")]

In [298]:
new_informators = new_informators[["инициалы", "ФИО", "region", "ditrict", "village", "порядковый номер информанта", "биография"]]

In [301]:
cur.executemany("INSERT INTO informators (code, name, current_region, current_district, current_village, old_id, bio) VALUES (%s, %s, %s, %s, %s, %s, %s)", new_informators.values.tolist())
con.commit()

In [575]:
dbinf = pd.read_sql_query("SELECT id, code, name, current_region FROM informators", con=con)

In [594]:
result = []
for i in range(df.shape[0]):
    row = df.iloc[i]
    
    for code in row["informators"]:
        code = code.strip().strip(".")
        if code == "???":
            continue
        infs = dbinf[dbinf["code"] == code]
        if infs.shape[0] != 1:
#             print(code)
            infs = infs[infs["current_region"] == row["region"]]
            if infs.shape[0] != 1:
                print(row["id"], row["region"], code, infs["current_region"].tolist(), infs.shape)
            else:
                result.append((row["id"], infs["id"].values[0]))
                
        else:
            result.append((row["id"], infs["id"].values[0]))

7669 Смоленская область СМТ [] (0, 4)
7670 Смоленская область СМТ [] (0, 4)
7671 Смоленская область СМТ [] (0, 4)
7672 Смоленская область СМТ [] (0, 4)
7673 Смоленская область СМТ [] (0, 4)
7674 Смоленская область СМТ [] (0, 4)
7675 Смоленская область СМТ [] (0, 4)
7676 Смоленская область СМТ [] (0, 4)
7677 Смоленская область СМТ [] (0, 4)
7678 Смоленская область СМТ [] (0, 4)
7679 Смоленская область СМТ [] (0, 4)
7680 Смоленская область СМТ [] (0, 4)
7681 Смоленская область СМТ [] (0, 4)
7682 Смоленская область СМТ [] (0, 4)
7683 Смоленская область СМТ [] (0, 4)
7684 Смоленская область СМТ [] (0, 4)
7685 Смоленская область СМТ [] (0, 4)
7686 Смоленская область СМТ [] (0, 4)
7687 Смоленская область СМТ [] (0, 4)
7725 Смоленская область ВА [] (0, 4)
7725 Смоленская область М [] (0, 4)
7751 Могилевская область КАТ [] (0, 4)
7992 Могилевская область Х [] (0, 4)
8329 Могилевская область С ['Могилевская область', 'Могилевская область'] (2, 4)
8730 Витебская область КПВ [] (0, 4)
9263 Витебс

In [595]:
cur.execute("SELECT id_text, id_informator FROM t_i WHERE id_text > 7294")
seen = set(cur.fetchall())

In [596]:
result = [(int(i[0]), int(i[1])) for i in result]

In [597]:
result = [i for i in result if i not in seen]
len(result)

2204

In [599]:
# result

In [600]:
cur.executemany("INSERT INTO t_i (id_text, id_informator) VALUES (%s, %s)", result)
con.commit()

In [314]:
# СМТ

# 7669, 7670, 7671, 7672, 7673, 7674, 7675, 7676, 7677, 7678, 7679, 7680, 7681, 7682, 7683, 7684, 7685, 7686, 7687 

# (7751, КАТ)
# (7992, Х)
# (8329, С)
# (8730, КПВ)
# (9263, КЕВ)

In [315]:
cur.execute("INSERT INTO t_i (id_text, id_informator) VALUES (%s, %s)", (7725, 466))
con.commit()

### География

In [384]:
geo = df[["region", "ditrict", "village"]].drop_duplicates().sort_values(by=["region", "ditrict", "village"])

In [386]:
db_villages = pd.read_sql_query("SELECT * FROM g_villages WHERE id < 48", con=con)

In [387]:
geo2 = geo.merge(db_villages, left_on=["village"], right_on=["village_name"], how="left")

In [388]:
data = geo2[geo2["id"].isna()][["region", "ditrict", "village"]].values.tolist()

In [389]:
# cur.executemany("INSERT INTO g_villages (map_region, map_district, village_name) VALUES (%s, %s, %s)", data)
# con.commit()

In [390]:
village_district = {
    "Кричев": "Кричевский район",
    "Мстиславль": "Мстиславский район",
    "Себеж": "Себежский район",
    "Велиж": "Велижский район"
}

In [391]:
geo["ditrict"].loc[geo[geo["ditrict"].isna()]["ditrict"].index] = geo[geo["ditrict"].isna()]["village"].apply(village_district.get)

In [392]:
cur.execute("SELECT region_name, id FROM g_regions")
geor = dict(cur.fetchall())

cur.execute("SELECT district_name, id FROM g_districts")
geod = dict(cur.fetchall())

cur.execute("SELECT village_name, id FROM g_villages")
geov = dict(cur.fetchall())

In [393]:
geo["reg"] = geo["region"].apply(geor.get)
geo["dis"] = geo["ditrict"].apply(geod.get)
geo["vil"] = geo["village"].apply(geov.get)

In [398]:
# cur.executemany("INSERT INTO g_geo_text (id_region, id_district, id_village) VALUES (%s, %s, %s)", geo[["reg", "dis", "vil"]].values.tolist())
# con.commit()

In [404]:
geo_text = pd.read_sql_query("SELECT id_village, id FROM g_geo_text", con=con)
geo_text

,id_village,id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
...,...,...
70,74,110
71,75,111
72,76,112
73,77,113


In [413]:
# df.merge(geo)

In [526]:
# geo

In [421]:
text_to_geo = df.merge(geo, how="left", left_on=["village"], right_on=["village"]).merge(geo_text, how="left", left_on=["vil"], right_on=["id_village"])[["id_y", "id_x"]]

In [423]:
cur.executemany("UPDATE texts SET geo_id = %s WHERE id = %s", text_to_geo.dropna().values.tolist())
con.commit()

In [424]:
text_to_geo.head()

,id_y,id_x
0,110,7295
1,110,7296
2,114,7297
3,114,7298
4,114,7299


### Ключевые слова

In [445]:
mapping = {
    "смерть": "жизнь-смерть",
    "нечистая сила": "демонология",
    "непогода": "погода",
    "угол дома": "угол",
    "прозвища": "прозвище",
    "предметы-обереги": "обереги",
    "потоп": "всемирный потоп",
    "коляда": "колядование",
    "сказки": "сказка",
    "новый": "новый-старый",
    "инородец": ["народ", "свой-чужой"], # 
    "птица": "птицы",
    "шитье": "шить",
    "новобрачные": "молодожены",
    "иноверец": ["вера", "свой-чужой"], # 
    "доля": "судьба",
    "гадание": "гадания",
    "булочка": "хлеб",
    "язык": "язык (орган)",
    "руки": "рука",
    "рыбы": "рыба",
    "терять–находить": "терять-находить",
    "кол": "колдун",
    "предания.": "предания",
    "книг": "книга",
    "молодой-ста": "молодой-старый",
    "конец петра и павла день": ["конец", "петра и павла день"],
    "животные": "животное",
    "михаил св.": "михаила день",
    "свадебный поезд": "поезд свадебный",
    "козел": "коза",
    "крещенье (календ.)": "крещенье (праздник)",
    "старый-новый": "новый-старый",
    "ласка": "ласка (зверь)",
    "легенда": "легенды",
    "освящать": "освящение",
    "застолье": "еда",
    "закрытый": "открытый-закрытый",
    "знамение": "знак-знамение",
    "правы-левый": "правый-левый",
    "границы": "граница",
    "хозяин": "хозяин-хозяйка",
    "умываться": "умывание"
}

In [446]:
cur.execute("SELECT word, id FROM keywords")
dbkeywords = {word.lower().strip(): i for word, i in cur.fetchall()}

In [448]:
result = []
for idx, row in df[["id", "keywords"]].values:
    for word in row:
        if word in dbkeywords:
            result.append((idx, dbkeywords[word]))
        elif word in mapping:
            if type(mapping[word]) == list:
                for w in mapping[word]:
                    result.append((idx, dbkeywords[w]))
            else:
                result.append((idx, dbkeywords[mapping[word]]))
        else:
            print(word)

воскрес
пещера
пещера
огурец
май
зелень
лопата

кожа

звон
муравейник
удавленник
полнолуние
кожа
смола
звон
звон
звон
звон
звон
звон


In [450]:
cur.executemany("INSERT INTO t_k (id_text, id_keyword) VALUES (%s, %s)", result)
con.commit()

### Доработка по информантам

In [452]:
df = pd.read_sql_query("SELECT * FROM informators WHERE id > 445", con=con)

In [465]:
from pymystem3 import Mystem
m = Mystem()

def g_name(name):
    global m
    name = re.sub("\(.*?\)", " ", name)
    res = m.analyze(name)
    fem, mal = 0, 0
    if len(res) == 6:
        res = res[2:]
    for i in res:
        if 'analysis' in i:
            try:
                if ',жен,' in i['analysis'][0]['gr']:
                    fem += 1
                elif ',муж,' in i['analysis'][0]['gr']:
                    mal += 1
            except:
                pass
    if fem > 0 and mal == 0:
        return 'f'
    elif fem == 0 and mal > 0:
        return 'm'
    else:
        return None

In [466]:
df["gender"] = df["name"].apply(g_name)

In [476]:
def get_year(bio):
    years = re.findall("([0-9]{4}) +г. *р.", bio[:20])
    if years:
        return int(years[0])
    else:
        return None

In [477]:
df["birth_year"] = df["bio"].apply(get_year)

In [497]:
geo_dict = dict({v: (d, r) for r, d, v in geo[["region", "ditrict", "village"]].values})

In [502]:
df["current_district"].loc[df["current_district"].isna()] = df["current_village"].loc[df["current_district"].isna()].apply(lambda x: geo_dict.get(x, [None, None])[0])

/home/dkbrz/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [513]:
import numpy as np

In [524]:
df2 = df[["gender", "birth_year", "current_district", "id"]]
df2 = df2.replace({np.nan: None})

In [525]:
cur.executemany(
    "UPDATE informators SET gender = %s, birth_year = %s, current_district = %s WHERE id = %s", 
    df2.values.tolist()
)
con.commit()

In [527]:
cur.execute("""SELECT 2015, texts.id FROM texts
JOIN g_geo_text ON g_geo_text.id = texts.geo_id
JOIN g_villages ON g_geo_text.id_village = g_villages.id
WHERE texts.id > 7294 AND texts.id < 9343 AND year = 2016""")
ids = cur.fetchall()

In [530]:
cur.executemany("UPDATE texts SET year = %s WHERE id = %s", ids)
con.commit()

In [533]:
cards = pd.read_csv("./data/undone/tblCards.csv").drop(["аннотация", "термины", "zvuk", "zvuk2", "photo", "photo2"], axis=1)
cards.head(1)

,текст,ключевые слова,программа,вопрос,село,год,информант,информант2,информант3,информант4,собиратель,собиратель2,собиратель3,собиратель4,номер
0,"[А что за старичок, к которому вы водили мужа?...","Ребенок, Демонология, Говорить, Болезнь, Медиц...",XIII,9 а,"Смоленская обл., Велижский р-н, Погорелье",2013,БТМ,NaN,NaN,NaN,НВП,АБМ,ОВБ,ПН,1


In [534]:
cards["txtlen"] = cards["текст"].apply(len)

In [536]:
cards.sort_values(by="txtlen", ascending=False)

,текст,ключевые слова,программа,вопрос,село,год,информант,информант2,информант3,информант4,собиратель,собиратель2,собиратель3,собиратель4,номер,txtlen
468,"[До этого рассказывали, что редко собираются, ...","Растения, Родство, Свой-чужой, Давать-брать, П...",ХXIa,"9 а, г, 2 а, б","Могилевская обл., Кричевский р-н, Ивановка",2014,АНИ,РАВ,КРИ,РНП,НВП,ОВБ,СНА,КВА,469,9966
411,[А на Крещение не гадали?]\r\n[ВТП:] Гада\ли.\...,"Пространство, Дом, Праздники, Время, Календарь...",IVa,"1 а, б, 8 б, а,","Будница, Велижский р-н, Смоленская обл.",2013,ВАМ,ВТП,ГЕА,NaN,АБМ,ОВБ,ВК,NaN,412,9836
850,"[ТММ сама рассказывает после того, как КНД рас...","Пространство, Праздники, Время, Календарь, Сем...",IIIa,"9 а, б","Могилевская обл., Кричевский р-н, Лобковичи",2014,КНД,ТММ,NaN,NaN,КВА,СНА,NaN,NaN,851,9723
854,"[После общих рассуждений, что везде свои обряд...","Пространство, Дом, Праздники, Время, Календарь...",IVa,"5 а, б","Могилевская обл., Кричевский р-н, Лобковичи",2014,КНД,ТММ,NaN,NaN,КВА,СНА,NaN,NaN,855,9714
464,"[До этого рассказывали про то, что в день отел...","Пространство, Дом, Запреты, Демонология, Тело ...",ХIII,"8 а, б, 9 а, г, д","Могилевская обл., Кричевский р-н, Ивановка",2014,АНИ,РАВ,КРИ,КТА,НВП,ОВБ,СНА,КВА,465,9697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1753,[На месте пожара нельзя строить дом. См. XV-12 б],"Пространство, Дом, Запреты, Огонь, Строительны...",IX,1 а,"Витебская обл., Россонский р-н, Хотьково",2015,КМС,NaN,NaN,NaN,АБМ,КВА,СНА,ОФ,1754,49
1978,[На Купалу люди умывались росой – см. XIII-9 а],"Праздники, Календарь, Лето, Иван Купала, Умывание",VIa,6 б,"Витебская обл., Россонский р-н, Амосенки",2016,РНЕ,РМФ,NaN,NaN,АБМ,НВП,NaN,NaN,1979,47
1955,[На Егорьев день выгоняли скот – см. VIII-14 a],"Праздники, Календарь, Скот, Скотоводство, Весн...",Va,12,"Витебская обл., Россонский р-н, Амосенки",2016,РМФ,РНЕ,NaN,NaN,АБМ,НВП,NaN,NaN,1956,47
1258,[Трехцветная кошка – хороший знак. См. IX-5 б],"Животное, Хороший-плохой, Приметы, Знак-знамен...",XVII,13 в,"Витебская обл., Россонский р-н, Краснополье",2015,АЛФ,NaN,NaN,NaN,АБМ,КВА,NaN,NaN,1259,46
